## Latest MOOC design analysis
#### Author: Hangxin Lu
#### Time: 16/01/2018

### Import libraries and set up database connections

In [37]:
import  psycopg2
import psycopg2.extras
import sys
import pprint
import pandas as pd

In [38]:
conn_string = "host='localhost' dbname='qua-kit'"
print ("Connecting to database\n	->%s" % (conn_string))
conn = psycopg2.connect(conn_string)
print ("Connected!\n")

Connecting to database
	->host='localhost' dbname='qua-kit'
Connected!



### Get necessary data from database

In [40]:
# all users
db_users = pd.read_sql_query ("SELECT * FROM \"user\"", conn)
# number of submission for each users
db_count_submissions = pd.read_sql_query("""SELECT author_id,COUNT(scenario.id) as n FROM SCENARIO GROUP BY author_id ORDER BY author_id""",conn)
# latest submission
db_scenarios_late = pd.read_sql_query("SELECT id, author_id, last_update, description, image, encode(geometry,'escape') as geo FROM scenario as a WHERE id = (SELECT max(b.id) FROM scenario as b where a.author_id = b.author_id) ORDER BY author_id", conn)

In [42]:
# all user actions
db_user_scenario_load =pd.read_sql_query( "SELECT * FROM user_scenario_load",conn)
db_user_scenario_update =pd.read_sql_query( "SELECT * FROM user_scenario_update",conn)
db_user_scenario_action =pd.read_sql_query( "SELECT * FROM user_scenario_action",conn)

In [53]:
# all cross feedback
db_criterion =pd.read_sql_query( "SELECT * FROM criterion",conn)
db_vote =pd.read_sql_query( "SELECT * FROM vote",conn)
db_peer_review =pd.read_sql_query( "SELECT * FROM review",conn)
db_expert_review = pd.read_sql_query("SELECT * FROM  expert_review",conn)
db_voter_score = pd.read_sql_query("SELECT * FROM vote_rating", conn)

In [56]:
# final grade and ratings
db_grade = pd.read_sql_query("SELECT * FROM current_scenario", conn)

In [57]:
db_grade

,id,history_scenario_id,author_id,task_id,description,grade,last_update,edx_grading_id
0,431,1,4,0,The proposal bases in the idea of the differen...,3.604098,2016-09-19 03:43:00.182149+02:00,788.0
1,924,2,6,0,,2.000000,2016-09-19 03:54:09.334886+02:00,867.0
2,401,14,47,0,"Gran Manzana, Mayor densidad al fondo, menor d...",3.663323,2016-09-19 14:56:55.798152+02:00,54.0
3,748,17,50,0,The organization plan proposed is based on the...,3.317773,2016-09-19 16:11:22.262375+02:00,856.0
4,677,19,54,0,,2.217234,2016-09-19 16:30:33.394324+02:00,500.0
5,757,25,62,0,,1.688535,2016-09-19 19:14:18.666579+02:00,839.0
6,860,27,73,0,The goal of this design is to promote a univer...,3.530680,2016-09-19 21:00:32.096023+02:00,777.0
7,656,31,2,0,,2.861820,2016-09-19 21:39:29.808677+02:00,501.0
8,829,42,58,0,Just getting started. Clearing out a central a...,1.150526,2016-09-20 00:54:32.851786+02:00,428.0
9,877,46,95,0,,3.264254,2016-09-20 04:12:31.637412+02:00,349.0


In [46]:
db_grade =pd.read_sql_query( "SELECT * FROM vote_grade",conn) #final score of student=ranking
db_rating =pd.read_sql_query( "SELECT author_id, AVG(value) as avg_rating from rating GROUP BY author_id ORDER by avg_rating DESC",conn)

DatabaseError: Execution failed on sql 'SELECT * FROM vote_grade': relation "vote_grade" does not exist
LINE 1: SELECT * FROM vote_grade
                      ^


In [48]:
# test one query result
db_review

,id,reviewer_id,scenario_id,criterion_id,positive,comment,timestamp
0,1,1,4,3,False,Thanks for the submission! Though it was not s...,2016-09-19 09:18:42.704570+02:00
1,2,69,20,3,True,,2016-09-19 20:03:51.844694+02:00
2,3,96,22,4,False,Interesting idea of the continous second insid...,2016-09-20 01:42:14.550422+02:00
3,4,103,21,4,False,Interesting proposal concerning the public spa...,2016-09-20 18:10:38.154039+02:00
4,5,103,1,3,False,Hello\r\nI like your design\r\n\r\nBut with 2 ...,2016-09-20 23:36:53.870185+02:00
5,6,103,17,3,True,Quite interesting\r\n\r\nI would make 2 sugges...,2016-09-21 00:46:08.500895+02:00
6,7,112,140,1,True,It is a good idea of yours to design an anti-n...,2016-09-22 06:33:51.358950+02:00
7,8,112,140,2,True,May be it could be better if you remove the se...,2016-09-22 06:37:26.678897+02:00
8,9,112,140,3,True,"Also, if you have designed social place in the...",2016-09-22 06:40:53.622331+02:00
9,10,112,144,1,True,Good work! \r\nHowever it should be organize...,2016-09-22 06:45:18.522933+02:00


In [ ]:
# drop database connection
conn.close()


### Format Dataframes